In [1]:
#We are testing a DL model for Mixed integer formulation for Water distribution network.
# The input data such as water demand and time-ahead dynamic energy prices are obtained from Singh et al (2019) paper.
# We computed the time ahead pump switching using MATALB Gurobi solver
# We obtained the time ahead pump switching for 2048 hours (85.33 days)
# We train_test the neural network in the ratio of 80-20.
# The nn is trained using ReLu and SoftMax (as the output is integer values)

#In our  case y is (z)

import numpy as np
import pandas as pd   #data analysis and manipulation library
import seaborn as sns #Python statistical data visualization library.
import matplotlib.pyplot as plt
from sklearn import model_selection #Tuning the hyperparameters, cross validation, model persistance are the important aspects of this.
from sklearn.model_selection import train_test_split #ML library to data analysis
from sklearn.metrics import accuracy_score, r2_score
import sklearn.metrics as metrics
from openpyxl import Workbook, load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows


from sklearn.decomposition import PCA #dimensionality reduction

%matplotlib inline

#Libraries for Deep learning framework from Keras

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import BatchNormalization

import keras
import keras.utils
from keras import utils as np_utils
from keras.models import Sequential

from tensorflow.keras import layers
from tensorflow.keras import utils as np_utils
from tensorflow.keras import models 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, LeakyReLU, PReLU, ELU #Import activation functions from keras.layers
from tensorflow.keras.layers import Dropout #Import dropout library, the objective of dropout is to prevent overfitting of data, However, we have to be careful while setting the dropout rate.
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler #Learning rate scheduler, Here call back is an object that can perform
                                                # at various actions at different stages of training 
from tensorflow.keras.callbacks import History #We can use the data collected from history to  [accuracy, loss, val_accuracy, val_loss]



import io #Deal with various types of input output
import os #Talk with os, like changing and modifying dirctories
import requests #Allow you to send HTTP/1.1, headers, form data, multipart files r = requests.get(‘https://github.com/timeline.json’)


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning) #Deprecated warning: it is a base category of warning about deprecated features


import dill
# dill.dump_session('notebook_env.db')




Using TensorFlow backend.


In [2]:
import io
#inputs = pd.read_csv(io.BytesIO(uploaded['1.csv']))

inputs=pd.read_csv(r'C:\Users\jyotib18\OPS_Data\2T12Ts5\feasible_input_log.csv')
inputs=inputs.dropna()


# Dataset is now stored in a Pandas Dataframe
inputs.shape

(193619, 75)

In [3]:
type(inputs)

pandas.core.frame.DataFrame

In [4]:
inputs.head

<bound method NDFrame.head of          170.36   184.93  0  1  1.1  0.1  3  0.2  1.2  1.3  ...  47.818  \
0       188.680   95.997  0  0    1    0  4    0    1    1  ...  40.997   
1        69.425   78.392  0  0    1    1  1    0    0    0  ...  39.418   
2       108.830   69.571  1  1    0    0  2    0    0    1  ...  42.353   
3       196.590  154.030  1  0    0    1  2    0    0    1  ...  30.652   
4       143.160  100.200  0  0    0    0  2    1    0    0  ...  34.814   
...         ...      ... .. ..  ...  ... ..  ...  ...  ...  ...     ...   
193614  105.530   44.108  0  0    1    1  2    0    0    1  ...  38.873   
193615  156.140  175.500  0  0    1    0  3    1    1    0  ...  46.673   
193616  164.760   48.545  0  1    0    0  4    1    1    1  ...  45.461   
193617  152.950  169.290  1  1    1    1  2    0    0    0  ...  43.121   
193618   93.005  164.140  1  1    0    0  3    1    1    0  ...  47.009   

        49.186  40.944  32.772  32.986   35.15  46.814  35.086  46.28

In [5]:
import io
outputs = pd.read_csv(r'C:\Users\jyotib18\OPS_Data\2T12Ts5\feasible_output_log.csv')
# Dataset is now stored in a Pandas Dataframe
outputs=outputs.dropna()
outputs.shape

(193619, 48)

In [6]:
outputs.head

<bound method NDFrame.head of         0   -0  -0.1  -0.2  -0.3  -0.4  -0.5  -0.6  -0.7    1  ...  -0.27  \
0       0  0.0   0.0   0.0   1.0   0.0   0.0   0.0   0.0  0.0  ...    0.0   
1       0 -0.0  -0.0  -0.0  -0.0  -0.0  -0.0   1.0  -0.0 -0.0  ...    0.0   
2       1 -0.0  -0.0  -0.0  -0.0  -0.0   1.0  -0.0   1.0 -0.0  ...    0.0   
3       1  0.0   0.0   0.0   1.0  -0.0  -0.0   1.0   0.0  0.0  ...    0.0   
4       0  1.0  -0.0  -0.0  -0.0   1.0  -0.0   1.0  -0.0 -0.0  ...    0.0   
...    ..  ...   ...   ...   ...   ...   ...   ...   ...  ...  ...    ...   
193614  0 -0.0  -0.0  -0.0   0.0   1.0   0.0   0.0   1.0  0.0  ...   -0.0   
193615  0  0.0   0.0   1.0   0.0   1.0   0.0   1.0   0.0  0.0  ...    0.0   
193616  0  1.0   1.0  -0.0   1.0  -0.0  -0.0  -0.0  -0.0 -0.0  ...   -0.0   
193617  1  0.0  -0.0   0.0   1.0  -0.0   0.0   1.0   0.0  0.0  ...   -0.0   
193618  1 -0.0   1.0  -0.0   1.0  -0.0  -0.0  -0.0  -0.0 -0.0  ...   -0.0   

        1.6  -0.28  -0.29  -0.30  -0.31  -0.3

In [7]:
x = inputs.values #converting data frame to array frame
x

array([[188.68 ,  95.997,   0.   , ...,  31.519,  34.798,  32.466],
       [ 69.425,  78.392,   0.   , ...,  38.714,  36.222,  48.468],
       [108.83 ,  69.571,   1.   , ...,  30.573,  39.798,  33.359],
       ...,
       [164.76 ,  48.545,   0.   , ...,  42.571,  37.691,  45.118],
       [152.95 , 169.29 ,   1.   , ...,  34.702,  41.056,  37.087],
       [ 93.005, 164.14 ,   1.   , ...,  43.464,  44.798,  37.136]])

In [8]:
type(outputs)

pandas.core.frame.DataFrame

In [9]:
y = outputs.values # converting dataframe "outputc" in array form i.e. 'y' is now in array form 
y # y is combination of power assignments and channel assignments both.

array([[ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0., -0., -0., ...,  0.,  0.,  0.],
       [ 1., -0., -0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  1.,  1., ..., -0., -0.,  0.],
       [ 1.,  0., -0., ...,  0.,  0.,  0.],
       [ 1., -0.,  1., ..., -0.,  1.,  0.]])

In [10]:
y.shape

(193619, 48)

In [11]:
x.shape

(193619, 75)

In [12]:
type(y)

numpy.ndarray

In [13]:
#y[0]

In [14]:
#y[0:50, 0] # First row and initial 50 columns of y 

In [15]:
# Splitting data for training and testing 
# Here both x and y are in array form (non-scaled)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=False, random_state=42) # old
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)

In [16]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [17]:
#Building neural network


model = tf.keras.Sequential()


model.add(Dense(20, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal')) # Input layer and 1st hidden layer
model.add(BatchNormalization())


#model.add(Dense(20, input_dim=x.shape[1], kernel_initializer='random_normal')) # Input layer and 1st hidden layer
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 1st hidden layer
#model.add(BatchNormalization())


model.add(Dense(20, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(Dense(100,  kernel_initializer='random_normal')) # 2nd Hidden layer
model.add(BatchNormalization())
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 2nd hidden layer


#model.add(Dense(40, kernel_initializer='random_normal')) # 3rd Hidden layer 
model.add(Dense(40, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 3rd hidden layer
model.add(BatchNormalization())


#model.add(Dense(60,  kernel_initializer='random_normal')) # 4th Hidden Layer 
model.add(Dense(60, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 4th hidden layer
model.add(BatchNormalization())

#model.add(Dense(80, kernel_initializer='random_normal')) # 5th Hidden Layer
model.add(Dense(80, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 5th hidden layer
model.add(BatchNormalization())

#model.add(Dense(100, kernel_initializer='random_normal')) # 6th Hidden layer
model.add(Dense(100, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 6th hidden layer
model.add(BatchNormalization())

#model.add(Dense(80, kernel_initializer='random_normal')) # 7th Hidden Layer 
model.add(Dense(80, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 4th hidden layer
model.add(BatchNormalization())


#model.add(Dense(60, kernel_initializer='random_normal')) # 7th Hidden Layer 
model.add(Dense(60, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 4th hidden layer
model.add(BatchNormalization())

model.add(Dense(40, input_dim=x.shape[1], activation='relu', kernel_initializer='random_normal'))
#model.add(Dense(40, kernel_initializer='random_normal')) # 7th Hidden Layer 
#model.add(LeakyReLU(alpha=0.1)) # Leaky Relu for 4th hidden layer
model.add(BatchNormalization())
model.add(Dense(y.shape[1], activation='sigmoid', kernel_initializer='random_normal')) # Output Layer 

In [18]:
x.shape[1]

75

In [19]:
# Compiling Neural network
'''
from tensorflow.keras.callbacks import EarlyStopping
bce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)

model.compile(loss=bce, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

#model.compile(optimizer='ADAM', loss= 'mean_squared_error' , metrics = ['accuracy']) 

monitor = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, 
                        verbose=1, mode='max', restore_best_weights=True)
'''

"\nfrom tensorflow.keras.callbacks import EarlyStopping\nbce = tf.keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.NONE)\n\nmodel.compile(loss=bce, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])\n\n#model.compile(optimizer='ADAM', loss= 'mean_squared_error' , metrics = ['accuracy']) \n\nmonitor = EarlyStopping(monitor='val_loss', min_delta=0.00001, patience=10, \n                        verbose=1, mode='max', restore_best_weights=True)\n"

In [20]:
from tensorflow.keras.callbacks import LearningRateScheduler
lrs = LearningRateScheduler(lambda x: 0.001)

In [21]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['mean_absolute_error']) 
#model.compile(optimizer='ADAM', loss='mean_squared_error', metrics = ['mean_absolute_error']) 
monitor = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10, 
                        verbose=1, mode='auto', restore_best_weights=True)

In [ ]:
# Fitting Neural network to training set

model_history=model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=512, verbose=2, callbacks=[monitor, lrs], epochs=1000)

Train on 174257 samples, validate on 19362 samples
Epoch 1/1000
174257/174257 - 5s - loss: 0.4522 - mean_absolute_error: 0.3183 - val_loss: 0.3651 - val_mean_absolute_error: 0.2351
Epoch 2/1000
174257/174257 - 3s - loss: 0.3053 - mean_absolute_error: 0.1905 - val_loss: 0.3036 - val_mean_absolute_error: 0.1849
Epoch 3/1000
174257/174257 - 3s - loss: 0.2891 - mean_absolute_error: 0.1796 - val_loss: 0.2901 - val_mean_absolute_error: 0.1769
Epoch 4/1000
174257/174257 - 2s - loss: 0.2827 - mean_absolute_error: 0.1757 - val_loss: 0.2821 - val_mean_absolute_error: 0.1742
Epoch 5/1000
174257/174257 - 3s - loss: 0.2788 - mean_absolute_error: 0.1736 - val_loss: 0.2783 - val_mean_absolute_error: 0.1729
Epoch 6/1000
174257/174257 - 3s - loss: 0.2763 - mean_absolute_error: 0.1723 - val_loss: 0.2756 - val_mean_absolute_error: 0.1721
Epoch 7/1000
174257/174257 - 3s - loss: 0.2740 - mean_absolute_error: 0.1712 - val_loss: 0.2736 - val_mean_absolute_error: 0.1701
Epoch 8/1000
174257/174257 - 3s - loss:

In [ ]:
y_train

In [ ]:
y_test = np.asarray(y_test)
x_test = np.asarray(x_test)
y_test


In [ ]:
y_test.shape

In [ ]:
# To evaluate model test loss
test_loss = model.evaluate(x_test, y_test)

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
# To plot training loss only

loss = model_history.history['loss']
epochs = range(1, len(loss)+1)
plt.plot(epochs, loss, 'ro', label='Training loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
# list all data in history

print(model_history.history.keys())
hfont = {'fontname':'Arial'}
# summarize history for accuracy
acc = model_history.history['mean_absolute_error']
val_acc = model_history.history['val_mean_absolute_error']

plt.plot(acc,color='red',linewidth=3)
plt.plot(val_acc,color='blue',linewidth=3)
#plt.title('model accuracy')
plt.ylabel('Mean Absolute Error', fontsize=16, **hfont)
plt.xlabel('Number of epochs', fontsize=16, **hfont)

#ylim(top=0.2)  # adjust the top leaving bottom unchanged
#ylim(bottom=0.05)  # adjust the bottom leaving top unchanged
plt.legend(['Training (ReLU), T=20', 'Testing (ReLU), T=20'], loc='upper right', prop={"size":14})
plt.ylim(top=0.185)  # adjust the top leaving bottom unchanged
plt.grid()
plt.savefig('accuracy_3_layer.pdf')
plt.show()

In [ ]:
# summarize history for loss

loss = model_history.history['loss']
val_loss = model_history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.savefig('loss_3_layer.pdf')
plt.grid()
plt.show()

In [ ]:
# Making the predictions for testing

# Predicting the Test set results
y_pred = model.predict(x_test) 
y_pred

In [ ]:
y_test.shape


In [ ]:
y_pred.shape

In [ ]:
# Evaluating the model # Test error

# Measure MSE error  
mse_test = metrics.mean_squared_error(y_test, y_pred)
print("Test Mean Squared error (MSE): {}".format(mse_test))

# Measure RMSE error 
rmse_test = np.sqrt(metrics.mean_squared_error(y_test, y_pred))
print("Test Root mean squared error (RMSE): {}".format(rmse_test))

# Measure MAE error
mae_test = metrics.mean_absolute_error(y_test, y_pred)
print("Test Mean absolute error (MAE): {}".format(mae_test))

In [ ]:
# Making the predictions for training

# Predicting the Training set results
y_pred_train = model.predict(x_train) 
y_pred_train

In [ ]:
y_pred_train.shape 

In [ ]:
# Evaluating the model # Training error

# Measure MSE error  
mse_train = metrics.mean_squared_error(y_train, y_pred_train)
print("Training Mean Squared error (MSE): {}".format(mse_train))

# Measure RMSE error 
rmse_train = np.sqrt(metrics.mean_squared_error(y_train, y_pred_train))
print("Training Root mean squared error (RMSE): {}".format(rmse_train))

# Measure MAE error
mae_train = metrics.mean_absolute_error(y_train, y_pred_train)
print("Training Mean absolute error (MAE): {}".format(mae_train))

In [ ]:
y_pred_flat = y_pred.flatten()
y_pred_flat

In [ ]:
y_test_flat = y_test.flatten()
y_test_flat

In [ ]:
# Regression score for test

r2_score_test = metrics.r2_score(y_test_flat, y_pred_flat)
print("R2 Score Test: {}".format(r2_score_test))

In [ ]:
y_pred_train_flat = y_pred_train.flatten()
y_pred_train_flat

In [ ]:
y_train_flat = y_train.flatten()
y_train_flat

In [ ]:
# Regression score for training

r2_score_train = metrics.r2_score(y_train_flat, y_pred_train_flat)
print("R2 Score Train: {}".format(r2_score_train))

In [ ]:
######################### Constraint verification for test data ##########################

In [ ]:
y_pred = model.predict(x_test)
y_pred

In [ ]:
x_test

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
y_pred

In [ ]:
#round-off the y_pred

import numpy as np
a_list = np.array(y_pred)
y_pred_binary = np.round(a_list)
print(y_pred_binary)



In [ ]:
y_pred_binary.shape

In [ ]:
y_pred_binary

In [ ]:
y_test

In [ ]:
len(y_pred_binary)

In [ ]:
inputs.shape

In [ ]:
x_test

In [ ]:
x_p = pd.DataFrame(x_test)
#pi = x_p.iloc[:, 27:50]
#pi
x_p.head

In [ ]:
#In the next cells, we are going to compute the suboptimality of the DNN solution vs others 

In [ ]:
x_pi = pd.DataFrame(x_test)
x_pi = x_pi.iloc[:, 19:31]
x_pi #Saving the energy prize

In [ ]:
#saving the test and predicted switching
y_test= pd.DataFrame(y_test)
y_train= pd.DataFrame(y_train)
y_pred= pd.DataFrame(y_pred)
y_pred


In [ ]:
'''
x_p = pd.DataFrame(inputs.values)            #Energy price of inputs
x_p = x_p.iloc[65544:81881, 18:50]            # Saving the Energy price
x_p.index = range(len(x_p))
x_p.columns = range(x_p.shape[1])


y_test= pd.DataFrame(y_test)
y_test = y_test.iloc[0:16377, 0:14]
y_test.index = range(len(y_test))
y_test.columns = range(y_test.shape[1])

y_pred= pd.DataFrame(y_pred_binary)
y_pred = y_pred.iloc[0:16377, 0:14]
y_pred.index = range(len(y_pred))
y_pred.columns = range(y_pred.shape[1])
'''
#testing = pd.concat([x_pi, y_test, y_pred], axis=1)
#testing.to_excel('testing.xlsx', index = False)

In [ ]:
concat_x_pi =  pd.concat([x_pi, x_pi, x_pi, x_pi], ignore_index=True, axis=1)
concat_x_pi= pd.DataFrame(concat_x_pi)
#concat_x_pi.index = new_index
concat_x_pi = concat_x_pi.T


In [ ]:
concat_x_pi = abs(concat_x_pi)
concat_x_pi.shape

In [ ]:
y_pred.shape


In [ ]:
result_predict = y_pred.dot(concat_x_pi);
result_predict.shape

In [ ]:
result_test = y_test.dot(concat_x_pi);
result_test.shape

In [ ]:
sub = abs(result_test - result_predict)


In [ ]:
sub = abs(result_test - result_predict)/(result_test)
sub

In [ ]:
suboptimality = np.diag(sub)
#plt.plot(suboptimality[48:72])
plt.xlabel('number of components')
plt.ylabel('element wise suboptimality')
plt.title('F*-F^/')
plt.axhline(y=0.15, color='r', linestyle='-')

In [ ]:
suboptimality.shape

In [ ]:
#Calculating the suboptimality for T=20, two tank model, batch normalization, activation = sigmoid, ReLU
#loss= binary cross entropy, pumps =4 
T = np.mean(suboptimality)
T